Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini 2.0 Chat Example

This notebook provides a simple example for interacting with Google's new Gemini 2.0 models using the unified Gen AI SDK. For more information please visit https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2

Install the needed dependencies for the notebook.

In [ ]:
!pip install -r requirements.txt

Import python libraries. Note the new "genai" library, which you can install using 'pip install google-genai'.

In [ ]:
import asyncio
import base64
import contextlib
import datetime
import os
import json
import wave
import itertools

import nest_asyncio

from IPython.display import display, Audio
from google import genai

Set your project variables. Change "YOUR_PROJECT_ID" to your GCP project ID.

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"

Initialize the model, specifying 'vertexai=True'. The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. 

In [ ]:
client = genai.Client(
    vertexai=True, project=project_id, location=region
)

Instead of performning an interactive chat session, you can optionally request a single turn generated response as outlined below.

In [ ]:
#response = client.models.generate_content(
#    model='gemini-2.0-flash-exp', contents='Hi Gemini!'
#)
#print(response.text)

## Start with a simple chat session

First, we'll select the model we want to use and then set the configuration the response for text only.

In [ ]:
model_id = "gemini-2.0-flash-exp"
config = {"response_modalities": ["TEXT"]}

We need to use nest_asyncio since we will be calling asyncio.run from within the already-running notebook's event loop.  The asyncio.run is used for running a coroutine directly on the event loop, but the event loop will already be running since we're calling it inside of a notebook. You probably will not need this outside of a Jupyter notebook environment. 

In [ ]:
nest_asyncio.apply()

Define an interactive chat session using the Gen AI API.

In [ ]:
async def interactive_chat(client, model_id, config):
    """
    Starts an interactive chat session with a specified model.

    Args:
        client: The client to use for communication.
        model_id: The ID of the model to interact with.
        config: The configuration to use for the session.
    """
    async with client.aio.live.connect(model=model_id, config=config) as session:
        while True:
            prompt = input("Enter your prompt here (or 'End' to quit): ")
            if prompt.lower() == "end":
                 break

            await session.send(prompt, end_of_turn=True)
            print("> You: ", prompt)  # Print the user's prompt

            async for response in session.receive():
                if response.text:
                    print(response.text, end="", flush=True)

Start the chat session by running the interactive_chat function. Simply type 'End' to stop the session.

In [ ]:
asyncio.run(interactive_chat(client, model_id, config))

 __________________________________________

That's how easy it is to use the new GenAI API